In [23]:
import pandas as pd


In [24]:
input_file_path = '/Users/thebekhruz/Desktop/100Days-Of-Code/100-Days-of-NLP-Odyssey/data/processed_data/processed_data_descr.csv'
output_file_path_titles = 'data/processed_data/processed_data_title.csv'
output_file_path_descriptions = 'data/processed_data/processed_data_descr.csv'

In [25]:
df = pd.read_csv(input_file_path, delimiter='\t', header=None, names=['doc_id', 'type', 'value'])
df.head(10)

,doc_id,type,value
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,description,east corner queen victoria road junction easto...
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,description,page bucks free press time capsule 7th july 18...
2,196c11e6-f7b6-392f-ae41-28653345087c,description,new police station high wycombe view opposite ...
3,7a5aace6-2398-3dcf-8843-37ff6ccea875,description,corridor entrance reference library
4,c66c4715-c03a-3aab-964b-e733f3ff1cf4,description,terrace brick flint cottage
5,d1159b13-8aa9-35c1-a4c2-fd13e24732b2,description,site dial house adjacent shop demolition easto...
6,e39a291d-ed39-3b56-b9aa-2022964a4114,description,row cottage wash house factory building cooper...
7,3b84ea4c-e194-3c34-abbf-064a41ad59da,description,60th anniversary opening library mock 1930sout...
8,bf418a27-d1d9-324b-a53f-50ab7ae8d81d,description,post office construction dial house christchur...
9,5bc83263-dcd0-3764-98cb-a3761480b4c7,description,new police station high wycombe


In [26]:
df.isna().value_counts()

doc_id  type   value
False   False  False    42300
               True         2
Name: count, dtype: int64

In [27]:
df.fillna('None', inplace=True)

In [28]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

In [32]:
df['DescriptionVector'] = df['value'].apply(lambda x: model.encode(x))

In [33]:
df.head()

,doc_id,type,value,DescriptionVector
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,description,east corner queen victoria road junction easto...,"[0.0010302312, -0.023901274, 0.01724779, 0.049..."
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,description,page bucks free press time capsule 7th july 18...,"[-0.01174476, 0.020760344, 0.0038109133, 0.004..."
2,196c11e6-f7b6-392f-ae41-28653345087c,description,new police station high wycombe view opposite ...,"[-0.002487674, 0.0084320465, 0.0564526, 0.0341..."
3,7a5aace6-2398-3dcf-8843-37ff6ccea875,description,corridor entrance reference library,"[0.00631654, 0.0067037106, -0.006011956, 0.020..."
4,c66c4715-c03a-3aab-964b-e733f3ff1cf4,description,terrace brick flint cottage,"[-0.057990633, -0.028714327, 0.003651726, 0.02..."


In [36]:
output_file_path = '/Users/thebekhruz/Desktop/100Days-Of-Code/100-Days-of-NLP-Odyssey/data/processed_data/processed_desc_vec.csv'
df.to_csv(output_file_path, sep='\t', index=False, header=None)

In [37]:
from elasticsearch import Elasticsearch

In [38]:
es = Elasticsearch(
    "https://localhost:9200/",
    basic_auth=("elastic", "XFvYbJBlk8zQARYN26ZV"),
    ca_certs="/Users/thebekhruz/Desktop/100Days-Of-Code/elasticsearch-8.12.1/config/certs/http_ca.crt"
)

es.ping()

True

In [43]:
mapping = {
    "properties": {
        "doc_id": {
            "type": "keyword"   # Suitable for unique identifiers
        },
        "description": {
            "type": "text"      # Suitable for full-text search
        },
        "description_vector": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}


In [44]:
es.indices.create(index='odyssey', mappings=mapping )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'odyssey'})

In [50]:
records = df.to_dict(orient='records')

In [55]:
# records[0]


In [56]:
for record in records:
    try:
        es.index(index='odyssey', body=record, id=record['doc_id'])
    except Exception as e:
        print(f'An error has occured: {e}')

In [60]:
input_query = 'Give me all documents which are related to queen victoria'
vectorised_query = model.encode(input_query)

query = {
    "field": "DescriptionVector",
    "query_vector": vectorised_query,
    "k": 5,
    "num_candidates":500
    

}

res = es.knn_search(index='odyssey', knn=query, source=["doc_id", "value"])
res['hits']['hits']

/var/folders/xh/x6c57wpx5hv7g4hxw7y4c6cc0000gn/T/ipykernel_8464/2144712460.py:13: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index='odyssey', knn=query, source=["doc_id", "value"])


[{'_index': 'odyssey',
  '_id': '30ad96fd-bf95-3b9c-9a02-e6c7a323aac7',
  '_score': 0.7996934,
  '_source': {'doc_id': '30ad96fd-bf95-3b9c-9a02-e6c7a323aac7',
   'value': 'appear press report image show queen accompany earl de la warr postmaster general leave queen horace harrison lady margaret hay lord cottesloe hon martin charteris'}},
 {'_index': 'odyssey',
  '_id': 'a32cf70d-a5f8-352f-aa4e-102e09af46cc',
  '_score': 0.7763544,
  '_source': {'doc_id': 'a32cf70d-a5f8-352f-aa4e-102e09af46cc',
   'value': 'upper green street high wycombe queen victoria plaque 1897 show picture victoria surround name country empire'}},
 {'_index': 'odyssey',
  '_id': 'fff8c4c3-f959-31ae-a5a4-2a99e4f3c95c',
  '_score': 0.7742809,
  '_source': {'doc_id': 'fff8c4c3-f959-31ae-a5a4-2a99e4f3c95c',
   'value': 'view plaque commemorate 60 year reign queen victoria queen great britain ireland empress india etc date 1897 set wall house upper green street'}},
 {'_index': 'odyssey',
  '_id': '184f1135-9585-3da0-95f

In [61]:
df.head()

,doc_id,type,value,DescriptionVector
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,description,east corner queen victoria road junction easto...,"[0.0010302312, -0.023901274, 0.01724779, 0.049..."
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,description,page bucks free press time capsule 7th july 18...,"[-0.01174476, 0.020760344, 0.0038109133, 0.004..."
2,196c11e6-f7b6-392f-ae41-28653345087c,description,new police station high wycombe view opposite ...,"[-0.002487674, 0.0084320465, 0.0564526, 0.0341..."
3,7a5aace6-2398-3dcf-8843-37ff6ccea875,description,corridor entrance reference library,"[0.00631654, 0.0067037106, -0.006011956, 0.020..."
4,c66c4715-c03a-3aab-964b-e733f3ff1cf4,description,terrace brick flint cottage,"[-0.057990633, -0.028714327, 0.003651726, 0.02..."
